In [186]:
# Exercise 1: train a trigram language model, i.e. 
# take two characters as an input to predict the 3rd one. 
# Feel free to use either counting or a neural net. 
# Evaluate the loss; Did it improve over a bigram model?

import torch
from itertools import product

In [187]:
words = open('names.txt', 'r').read().splitlines()

In [188]:
chars = set(''.join(words))
chars.add('.')
chars = sorted(chars)
combs = sorted([''.join(comb) for comb in product(chars, repeat=2)])

stoi = {s:i for i,s in enumerate(chars)}
ctoi = {c:i for i,c in enumerate(combs)}

itos = {s:i for i,s in stoi.items()}
itoc = {c:i for i,c in ctoi.items()}

In [189]:
xs, ys = [], []

for w in words[:1]:
    end = len(w) - 1
    combs = ['..'] + ['.' + w[0]] + [w[i] + w[i+1] for i in range(end)] + [w[end] + '.']
    print(combs)

    for comb, ch in zip(combs, combs[1:]):
        print(comb, ch[1])
        row = ctoi[comb]
        col = stoi[ch[1]]
        xs.append(row)
        ys.append(col)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()

print(xs)
print(ys)

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((729, 27), generator=g, requires_grad=True)

['..', '.e', 'em', 'mm', 'ma', 'a.']
.. e
.e m
em m
mm a
ma .
tensor([  0,   5, 148, 364, 352])
tensor([ 5, 13, 13,  1,  0])


In [190]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=729).float()
xenc.shape

torch.Size([5, 729])

In [200]:
for k in range(100):
    xenc = F.one_hot(xs, num_classes=729).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(dim=1, keepdim=True)
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())
    
    W.grad = None
    loss.backward()
    W.data += -1 * W.grad


0.007134473416954279
0.007123799528926611
0.007113209925591946
0.007102536968886852
0.007091984152793884
0.007081382907927036
0.007070890162140131
0.007060445845127106
0.00704996520653367
0.007039508316665888
0.007029184140264988
0.007018847856670618
0.0070084393955767155
0.006998151540756226
0.006987899541854858
0.006977671291679144
0.006967479828745127
0.0069572762586176395
0.006947144865989685
0.006937037222087383
0.006926918867975473
0.006916834972798824
0.006906835827976465
0.006896813400089741
0.006886885967105627
0.006876851432025433
0.0068669975735247135
0.00685708224773407
0.0068472521379590034
0.006837374065071344
0.006827604025602341
0.006817785557359457
0.006808075122535229
0.006798353977501392
0.006788596510887146
0.006778995040804148
0.006769356783479452
0.006759719457477331
0.006750178523361683
0.006740637123584747
0.006731107831001282
0.006721530109643936
0.006712096743285656
0.00670264009386301
0.006693220231682062
0.006683822721242905
0.006674474570900202
0.0066651380

In [201]:
print(probs[torch.arange(5), ys[:5]])

tensor([0.9939, 0.9938, 0.9938, 0.9939, 0.9938], grad_fn=<IndexBackward0>)


In [202]:
# Sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5): 
    out = []
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=729).float()
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(dim=1, keepdim=True)

        old_comb = itoc[ix]
        ix_ch = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        ch = itos[ix_ch]
        comb = old_comb[1] + ch
        ix = ctoi[comb]

        out.append(itoc[ix][1])
        if itoc[ix][1] == '.':
            break
    print(''.join(out))

emma.
emma.
emma.
emma.
emma.
